## Notebook for working with Clever data






In [ ]:
storage_account = 'sttesteduanalyticscisd3'

stage1 = f'abfss://stage1@{storage_account}.dfs.core.windows.net'
stage2 = f'abfss://stage2@{storage_account}.dfs.core.windows.net'
stage3 = f'abfss://stage3@{storage_account}.dfs.core.windows.net'

In [ ]:
# Process personalized_instruction_by_lesson_math.csv
df = spark.read.csv(stage1 + '/clever', header='true', inferSchema='true')
df = df.withColumn('sis_id',df.sis_id.cast('string'))
df.write.format('parquet').mode('overwrite').save(f'{stage2}/clever/resource_usage_students')
df.show()

In [ ]:
# Anonymize data and load into stage3
from pyspark.sql.functions import sha2, lit
df = spark.read.format('parquet').load(f'{stage2}/clever/resource_usage_students')
df = df.withColumn('sis_id', sha2(df.sis_id, 256)).withColumn('clever_user_id',lit('*')).withColumn('clever_school_id',lit('*'))
df.write.format('parquet').mode('overwrite').save(f'{stage3}/clever/resource_usage_students')


In [ ]:
# Create sql on-demand db's for Clever data

def create_spark_db(db_name, source_path):
    spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
    spark.sql(f"create table if not exists {db_name}.comprehensive_student_lesson_activity_with_standards_ela using PARQUET location '{source_path}/resource_usage_students'")

# Drop all tables in a db, then drop the db
def drop_db(db_name):
    df = spark.sql('SHOW TABLES FROM ' + db_name)
    for row in df.rdd.collect():
        spark.sql(f"DROP TABLE IF EXISTS {db_name}.{row['tableName']}")
    spark.sql(f"DROP DATABASE {db_name}")    

#drop_db('s2_iready')
create_spark_db('s2_clever', f'{stage2}/clever')
#drop_db('s3_iready')
create_spark_db('s3_clever', f'{stage3}/clever')